In [3]:
import cv2
import numpy as np
import mediapipe as mp
import os
from math import hypot

# Initialize MediaPipe 
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Volume tool (macOS only)
def set_volume_mac(level):
    level = int(max(0, min(100, level)))
    os.system(f"osascript -e 'set volume output volume {int(level)}'")
# Brightness tool (macOS only)
def set_brightness_mac(level):
    level = max(0.1, min(1.0, level))
    os.system(f"brightness {level}")

# Helper for distance
def distance(p1, p2):
    return hypot(p2[0] - p1[0], p2[1] - p1[1])

# Smoothing
def smooth(val, prev, alpha=0.3):
    return alpha * val + (1 - alpha) * prev

# Toggle state
hand_states = {
    "Left": {"enabled": True, "prev_toggle": False, "prev_val": 0.5},
    "Right": {"enabled": True, "prev_toggle": False, "prev_val": 50}
}

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    if not success:
        break

    img = cv2.flip(img, 1)  # Mirror image
    h, w, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    if results.multi_hand_landmarks and results.multi_handedness:
        for hand_landmarks, hand_type in zip(results.multi_hand_landmarks, results.multi_handedness):
            label = hand_type.classification[0].label  # 'Left' or 'Right'
            lm_list = [(int(lm.x * w), int(lm.y * h)) for lm in hand_landmarks.landmark]

            mp_draw.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            cv2.putText(img, f'{label} Hand{"[ON]" if hand_states[label]["enabled"] else "[OFF]"}',
                        (10, 30 if label == "Left" else 70 ),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0),2)
            
            if len(lm_list)>12:
                toggle_dist = distance(lm_list[4], lm_list[12])
                toggle = toggle_dist < 40
                if toggle and not hand_states[label] ["prev_toggle"]:
                    hand_states[label]["enabled"] = not hand_states[label] ["enabled"]
                hand_states[label]["prev_toggle"] = toggle
            
            if not hand_states[label]["enabled"]:
                continue

            if len(lm_list) > 8:
                x1, y1 = lm_list[4]
                x2, y2 = lm_list[8]
                length = distance((x1, y1), (x2, y2))

                cv2.circle(img, (x1, y1), 10, (0, 0, 255), cv2.FILLED)
                cv2.circle(img, (x2, y2), 10, (0, 0, 255), cv2.FILLED)
                cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 2)

            if label == "Left":
                brightness = np.interp(length, [30, 200], [0.1, 1.0])
                brightness = smooth(brightness, hand_states[label]["prev_val"])
                if abs(brightness - hand_states[label]["prev_val"]) > 0.05:
                    set_brightness_mac(brightness)
                    hand_states[label]["prev_val"] = brightness
                cv2.putText(img, f'Brightness: {brightness:.2f}', (10, 110),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
                
            if len(lm_list) > 8:
                x1, y1 = lm_list[4]  # Thumb tip
                x2, y2 = lm_list[8]  # Index tip
                length = distance((x1, y1), (x2, y2))

                cv2.circle(img, (x1, y1), 10, (0, 0, 255), cv2.FILLED)
                cv2.circle(img, (x2, y2), 10, (0, 0, 255), cv2.FILLED)
                cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 2)

                if label == "Left":
                    # Brightness range: 0.1 to 1.0
                    brightness = np.interp(length, [30, 200], [0.1, 1.0])
                    brightness = smooth(brightness, hand_states[label]["prev_val"])
                    if abs(brightness - hand_states[label]["prev_val"]) > 0.05:
                      set_brightness_mac(brightness)
                      hand_states[label]["prev_val"] = brightness
                    cv2.putText(img, f'Brightness: {brightness:.2f}', (10, 110),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

                elif label == "Right":
                    # Volume range: 0 to 100
                    volume = np.interp(length, [30, 200], [0, 100])
                    volume = smooth(volume, hand_states[label]["prev_val"])
                    if abs(volume - hand_states[label]["prev_val"]) > 2:
                      set_volume_mac(volume)
                      hand_states[label]["prev_val"] = volume
                    cv2.putText(img, f'Volume: {int(volume)}%', (10, 150),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 255), 2)

    cv2.imshow("Gesture Control (macOS)", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1747933158.621577    9066 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1747933158.630213  190240 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747933158.634642  190249 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
brightness: failed to set brightness of display 0x1 (error -536870201)
brightness: failed to set brightness of display 0x1 (error -536870201)
brightness: failed to set brightness of display 0x1 (error -536870201)
brightness: failed to set brightness of display 0x1 (error -536870201)
brightness: failed to set brightness of display 0x1 (error -536870201)
brightness: failed to set brightness of display 0x1 (error -536870201)
brightness: failed to set brightness of display 0x1 (error -536870201)
brightness: failed to s